In [6]:
!echo foo > .gitignore

In [9]:
!git cl

Nothing specified, nothing added.
Maybe you wanted to say 'git add .'?


In [5]:
!git status

On branch master

No commits yet

Untracked files:
  (use "git add <file>..." to include in what will be committed)

	.config/
	sample_data/

nothing added to commit but untracked files present (use "git add" to track)


In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models

import numpy as np

import PIL

import sys
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler

from torch.utils import data

import matplotlib.pyplot as plt
import time
import logging
import datetime

cuda = torch.cuda.is_available()
cuda

True

In [ ]:
! nvidia-smi


Fri Nov  6 05:13:40 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.51.06    Driver Version: 450.51.06    CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            On   | 00000000:00:1E.0 Off |                    0 |
| N/A   46C    P8    10W /  70W |      3MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
class AddGaussianNoise(object):
    def __init__(self, mean=0., std=1.):
        self.std = std
        self.mean = mean
        
    def __call__(self, tensor):
      res = tensor + torch.randn(tensor.size()) * self.std + self.mean
      return torch.clamp(input=res, min=0.0, max=1.0)
    
    def __repr__(self):
        return self.__class__.__name__ + '(mean={0}, std={1})'.format(self.mean, self.std)

In [ ]:
def train_epoch(model, train_loader, criterion, optimizer):
    model.train()

    running_loss = 0.0
    total_predictions = 0.0
    correct_predictions = 0.0
    
    for batch_idx, (data, target) in enumerate(train_loader):   
        optimizer.zero_grad()   # .backward() accumulates gradients
        data = data.to(device)
        target = target.to(device) # all data & model on same device
        
        with torch.cuda.amp.autocast():
          outputs = model(data)
          loss = criterion(outputs, target)
          running_loss += loss.item()

          _, predicted = torch.max(outputs.data, 1)
          total_predictions += target.size(0)
          correct_predictions += (predicted == target).sum().item()
        
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()


        del data
        del target
    
    
    running_loss /= len(train_loader)
    acc = (correct_predictions/total_predictions)*100.0
    logging.info('Training Loss: {}'.format(running_loss))
    logging.info('Training Accuracy: {}%'.format(acc))
    return running_loss


def test_model(model, test_loader, criterion):
    with torch.no_grad():
        model.eval()

        running_loss = 0.0
        total_predictions = 0.0
        correct_predictions = 0.0

        for batch_idx, (data, target) in enumerate(test_loader): 
            #print(data.shape)  
            data = data.to(device)
            target = target.to(device)

            with torch.cuda.amp.autocast():
              outputs = model(data)

              _, predicted = torch.max(outputs.data, 1)
              total_predictions += target.size(0)
              correct_predictions += (predicted == target).sum().item()

              loss = criterion(outputs, target).detach()
              running_loss += loss.item()

            del data
            del target

        running_loss /= len(test_loader)
        acc = (correct_predictions/total_predictions)*100.0
        logging.info('Testing Loss: {}'.format(running_loss))
        logging.info('Testing Accuracy: {}%'.format(acc))
        return running_loss, acc

def init_weights(m):
    if type(m) == nn.Conv2d or type(m) == nn.Linear:
        torch.nn.init.xavier_normal_(m.weight.data)

In [ ]:
# configure logging
logger = logging.getLogger("")

# reset handler
for handler in logging.root.handlers[:]:
  logging.root.removeHandler(handler)

# set handler
stream_hdlr = logging.StreamHandler()
file_hdlr = logging.FileHandler('/home/ubuntu/project/log_{}.log'.format(datetime.datetime.now()))

formatter = logging.Formatter('%(asctime)s %(levelname)s %(message)s')
stream_hdlr.setFormatter(formatter)
file_hdlr.setFormatter(formatter)

logger.addHandler(stream_hdlr)
logger.addHandler(file_hdlr)

logger.setLevel(logging.INFO)

In [ ]:
class VGG16(nn.Module):
    def __init__(self, nb_classes=10):
        super(VGG16, self).__init__()
        self.vgg16 = models.vgg16_bn()
        self.linear = nn.Linear(1000, nb_classes)
        
    def forward(self, x):
        x = self.vgg16(x)
        x = self.linear(x)
        return x

In [ ]:
train_batchsize = 128
test_batchsize = 100
num_workers = 8
num_classes = 10

n_epochs = 25
img_size = 224
lr = 1e-4
min_lr = 1e-8
weight_decay = 5e-4
num_models = 30
noise_std = 0.1

device = torch.device("cuda" if cuda else "cpu")

hyper_params = {'lr': lr, 'min_lr': min_lr, 'weight_decay': weight_decay, 'num_models': num_models, 'num_epochs': n_epochs, 'noise_std': noise_std}
logging.info(hyper_params)


2020-11-06 05:14:15,847 INFO {'lr': 0.0001, 'min_lr': 1e-08, 'weight_decay': 0.0005, 'num_models': 30, 'num_epochs': 25, 'noise_std': 0.1}


In [ ]:
seeds = np.arange(30, 45)

for seed in seeds:
  logging.info('Seed: {}'.format(seed))
  torch.manual_seed(seed)
  

  train_transform = transforms.Compose([transforms.Resize(size=img_size),
                      transforms.ColorJitter(hue=.05, saturation=.05),
                      transforms.RandomHorizontalFlip(p=0.5),
                      transforms.RandomRotation(20, resample=PIL.Image.BILINEAR),
                      transforms.ToTensor(),
                      transforms.Normalize((0, 0, 0), (1, 1, 1)),
                      AddGaussianNoise(0., noise_std)])

  test_transform = transforms.Compose([transforms.Resize(size=img_size),
                     transforms.ToTensor(),
                     transforms.Normalize((0, 0, 0), (1, 1, 1)),
                     AddGaussianNoise(0., noise_std)])

  
  trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=train_transform)
  testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=test_transform)
  trainloader = torch.utils.data.DataLoader(trainset, batch_size=train_batchsize, shuffle=True, num_workers=num_workers)
  testloader = torch.utils.data.DataLoader(testset, batch_size=test_batchsize, shuffle=False, num_workers=num_workers)

  # model = models.vgg16()
  model = VGG16(num_classes)
  model.apply(init_weights)
  criterion = nn.CrossEntropyLoss()
  # optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9, nesterov=False)
  optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
  scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=1, cooldown=5, min_lr=min_lr, verbose=True)
  scaler = torch.cuda.amp.GradScaler()
  model.to(device)

  # Train_loss = []
  # Test_loss = []
  # Test_acc = []

  for i in range(n_epochs):
    tic = time.time()
    logging.info('Epoch: {}'.format(i))
    train_loss = train_epoch(model, trainloader, criterion, optimizer)
    test_loss, test_acc = test_model(model, testloader, criterion)
    scheduler.step(test_loss)
    # Train_loss.append(train_loss)
    # Test_loss.append(test_loss)
    # Test_acc.append(test_acc)
    toc = time.time()
    logging.info('Time: {}s'.format(toc - tic))
    logging.info('='*20)
    
  torch.save({'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'scheduler_state_dict': scheduler.state_dict()}, 
        "/home/ubuntu/project/Model_{}_{}".format(seed, noise_std))
  
  torch.cuda.empty_cache()
  del model
  del criterion
  del optimizer
  del scheduler
  del scaler
  del trainloader
  del testloader
  del train_transform
  del test_transform

  


2020-11-06 05:14:57,166 INFO Seed: 30


Files already downloaded and verified
Files already downloaded and verified


2020-11-06 05:15:04,296 INFO Epoch: 0
2020-11-06 05:23:54,602 INFO Training Loss: 2.209992938944141
2020-11-06 05:23:54,604 INFO Training Accuracy: 31.240000000000002%
2020-11-06 05:24:25,429 INFO Testing Loss: 1.472707427740097
2020-11-06 05:24:25,430 INFO Testing Accuracy: 45.65%
2020-11-06 05:24:25,431 INFO Time: 561.1347258090973s
2020-11-06 05:24:25,432 INFO ====================
2020-11-06 05:24:25,432 INFO Epoch: 1
2020-11-06 05:33:21,818 INFO Training Loss: 1.5236877218231826
2020-11-06 05:33:21,819 INFO Training Accuracy: 44.114%
2020-11-06 05:33:52,522 INFO Testing Loss: 1.3250112450122833
2020-11-06 05:33:52,523 INFO Testing Accuracy: 51.629999999999995%
2020-11-06 05:33:52,524 INFO Time: 567.0918924808502s
2020-11-06 05:33:52,524 INFO ====================
2020-11-06 05:33:52,525 INFO Epoch: 2
2020-11-06 05:42:49,494 INFO Training Loss: 1.283415891477824
2020-11-06 05:42:49,496 INFO Training Accuracy: 53.598%
2020-11-06 05:43:20,434 INFO Testing Loss: 1.1734548163414003
2020-

Epoch     8: reducing learning rate of group 0 to 1.0000e-05.


2020-11-06 06:39:37,065 INFO Training Loss: 0.5374963569366719
2020-11-06 06:39:37,066 INFO Training Accuracy: 81.736%
2020-11-06 06:40:07,876 INFO Testing Loss: 0.4743792320787907
2020-11-06 06:40:07,878 INFO Testing Accuracy: 83.91%
2020-11-06 06:40:07,878 INFO Time: 565.4552090167999s
2020-11-06 06:40:07,879 INFO ====================
2020-11-06 06:40:07,879 INFO Epoch: 9
2020-11-06 06:49:05,119 INFO Training Loss: 0.4915067335528791
2020-11-06 06:49:05,121 INFO Training Accuracy: 83.262%
2020-11-06 06:49:36,085 INFO Testing Loss: 0.44659701600670815
2020-11-06 06:49:36,086 INFO Testing Accuracy: 85.13%
2020-11-06 06:49:36,087 INFO Time: 568.2074685096741s
2020-11-06 06:49:36,088 INFO ====================
2020-11-06 06:49:36,088 INFO Epoch: 10
2020-11-06 06:58:34,841 INFO Training Loss: 0.4712519364436264
2020-11-06 06:58:34,842 INFO Training Accuracy: 84.02%
2020-11-06 06:59:05,863 INFO Testing Loss: 0.44565596073865893
2020-11-06 06:59:05,864 INFO Testing Accuracy: 85.02%
2020-11-0

...

2020-11-06 16:30:34,792 INFO Training Loss: 0.293797155322931
2020-11-06 16:30:34,793 INFO Training Accuracy: 89.956%
2020-11-06 16:31:05,697 INFO Testing Loss: 0.34311083823442456
2020-11-06 16:31:05,698 INFO Testing Accuracy: 88.52%
2020-11-06 16:31:05,699 INFO Time: 570.4010057449341s
2020-11-06 16:31:05,700 INFO ====================
2020-11-06 16:31:05,700 INFO Epoch: 21
2020-11-06 16:40:04,376 INFO Training Loss: 0.2809066207665007
2020-11-06 16:40:04,377 INFO Training Accuracy: 90.322%
2020-11-06 16:40:35,201 INFO Testing Loss: 0.35035929262638094
2020-11-06 16:40:35,202 INFO Testing Accuracy: 88.31%
2020-11-06 16:40:35,203 INFO Time: 569.5029788017273s
2020-11-06 16:40:35,204 INFO ====================
2020-11-06 16:40:35,205 INFO Epoch: 22
2020-11-06 16:49:39,184 INFO Training Loss: 0.27509108772668084
2020-11-06 16:49:39,185 INFO Training Accuracy: 90.556%
2020-11-06 16:50:10,214 INFO Testing Loss: 0.3421341433376074
2020-11-06 16:50:10,216 INFO Testing Accuracy: 88.72%
2020-11

Files already downloaded and verified
Files already downloaded and verified


2020-11-06 17:09:25,945 INFO Epoch: 0
2020-11-06 17:18:28,001 INFO Training Loss: 2.2237036377572648
2020-11-06 17:18:28,002 INFO Training Accuracy: 30.278%
2020-11-06 17:18:58,881 INFO Testing Loss: 1.501912410259247
2020-11-06 17:18:58,882 INFO Testing Accuracy: 45.7%
2020-11-06 17:18:58,883 INFO Time: 572.9383342266083s
2020-11-06 17:18:58,884 INFO ====================
2020-11-06 17:18:58,884 INFO Epoch: 1
2020-11-06 17:28:00,715 INFO Training Loss: 1.5028310358676764
2020-11-06 17:28:00,716 INFO Training Accuracy: 45.385999999999996%
2020-11-06 17:28:31,813 INFO Testing Loss: 1.2620339369773865
2020-11-06 17:28:31,814 INFO Testing Accuracy: 55.31%
2020-11-06 17:28:31,815 INFO Time: 572.9308850765228s
2020-11-06 17:28:31,816 INFO ====================
2020-11-06 17:28:31,817 INFO Epoch: 2
2020-11-06 17:37:33,687 INFO Training Loss: 1.269964002739743
2020-11-06 17:37:33,688 INFO Training Accuracy: 54.55799999999999%
2020-11-06 17:38:04,605 INFO Testing Loss: 1.0615398675203322
2020-11

Epoch    11: reducing learning rate of group 0 to 1.0000e-05.


2020-11-06 19:03:18,227 INFO Training Loss: 0.4271228654152902
2020-11-06 19:03:18,229 INFO Training Accuracy: 85.422%
2020-11-06 19:03:49,179 INFO Testing Loss: 0.39600149348378183
2020-11-06 19:03:49,181 INFO Testing Accuracy: 86.77%
2020-11-06 19:03:49,181 INFO Time: 573.8946163654327s
2020-11-06 19:03:49,182 INFO ====================
2020-11-06 19:03:49,183 INFO Epoch: 12
2020-11-06 19:12:52,133 INFO Training Loss: 0.38604037391255275
2020-11-06 19:12:52,134 INFO Training Accuracy: 86.91%
2020-11-06 19:13:23,129 INFO Testing Loss: 0.38537114098668096
2020-11-06 19:13:23,130 INFO Testing Accuracy: 87.11%
2020-11-06 19:13:23,131 INFO Time: 573.9478902816772s
2020-11-06 19:13:23,131 INFO ====================
2020-11-06 19:13:23,132 INFO Epoch: 13
2020-11-06 19:22:19,794 INFO Training Loss: 0.360413967000554
2020-11-06 19:22:19,795 INFO Training Accuracy: 87.78%
2020-11-06 19:22:50,438 INFO Testing Loss: 0.37431432381272317
2020-11-06 19:22:50,439 INFO Testing Accuracy: 87.68%
2020-11-

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# functions to show an image


def imshow(img):
    img = img + 1 / 2     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


# get some random training images
dataiter = iter(trainloader)
images, labels = dataiter.next()

# show images
imshow(torchvision.utils.make_grid(images))
# print labels
print(' '.join('%5s' % classes[labels[j]] for j in range(4)))

In [ ]:
  torch.save({'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'scheduler_state_dict': scheduler.state_dict()}, 
        "/home/ubuntu/project/Model_{}".format(35))